In [1]:
import pandas as pd

# Define the correct data types for the columns
dtype_dict = {
    'nr': str,
    'nr2': str,
    'type': str,
    'star_statement': str,
    'reviewer_nick': str,
    'order': str,
    'statement': str,
    'name_doc': str,
    'star_doc': str,
    'rev_nr': str
}

# Read the Excel file with specified data types
file_path = './stimuli_l2.xlsx'  # Replace with the actual file path
df = pd.read_excel(file_path, dtype=dtype_dict)

# Group the rows by doctor (nr, doctor_name, rev_nr, order)
grouped = df.groupby(['nr', 'name_doc', 'rev_nr', 'order', 'star_doc'])

# Generate the TypeScript array of objects
ts_objects = f"""
import type {{ IDoctorObjectL2 }} from "$lib/interfaces/IDoctor";
/*
Automatically generated stimuli data from the Excel sheet for level 2
*/
export const doctorsL2: IDoctorObjectL2[] = [\n"""

# Loop through each group and create TypeScript object for each doctor
for (nr, doctor_name, rev_nr, order, star_doc), group in grouped:
    # Create a list of reviews for this doctor
    reviews_list = []
    for _, row in group.iterrows():
        review = f"""    {{
        nr: "{row['nr']}",
        nr2: "{row['nr2']}",
        star_statement: "{row['star_statement']}",
        statement: "{row['statement']}",
        reviewer_nick: "{row['reviewer_nick']}",
        type: "{row['type']}"
    }}"""
        reviews_list.append(review)
    
    # Convert the list of reviews into a single string
    reviews_str = ",\n".join(reviews_list)
    
    # Add the doctor object with its reviews
    ts_objects += f"""  {{
    nr: "{nr}",
    doctor_name: "{doctor_name}",
    rev_nr: "{rev_nr}",
    order: "{order}",
    star_doc: "{star_doc}",
    reviews: [
{reviews_str}
    ]
  }},
"""

ts_objects += "];"

# Define the path to save the TypeScript file
output_file_path = '../src/lib/data/doctorsL2.ts'

# Write the TypeScript constant object array to the file
with open(output_file_path, 'w') as file:
    file.write(ts_objects)

print(f"Data has been written to {output_file_path}")


Data has been written to ../src/lib/data/doctorsL2.ts
